Write a program to implement neural network on SONAR dataset compare the result of :

1)Backpropagation neural network with adam optimizer

2) L1 and L2 regularization

3)  Early stopping with p=5

4)Dropout = 0.2 for input layer and 2nd hidden layer

No of hidden layers are 2 
Also note time before model.fit and after it

In [73]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import time

In [74]:
data=pd.read_csv('/content/drive/MyDrive/ML_csvs/sonar.csv')
data

0.0200  0.0371  0.0428  0.0207  0.0954  0.0986  0.1539  0.1601  0.3109  \
0    0.0453  0.0523  0.0843  0.0689  0.1183  0.2583  0.2156  0.3481  0.3337   
1    0.0262  0.0582  0.1099  0.1083  0.0974  0.2280  0.2431  0.3771  0.5598   
2    0.0100  0.0171  0.0623  0.0205  0.0205  0.0368  0.1098  0.1276  0.0598   
3    0.0762  0.0666  0.0481  0.0394  0.0590  0.0649  0.1209  0.2467  0.3564   
4    0.0286  0.0453  0.0277  0.0174  0.0384  0.0990  0.1201  0.1833  0.2105   
..      ...     ...     ...     ...     ...     ...     ...     ...     ...   
202  0.0187  0.0346  0.0168  0.0177  0.0393  0.1630  0.2028  0.1694  0.2328   
203  0.0323  0.0101  0.0298  0.0564  0.0760  0.0958  0.0990  0.1018  0.1030   
204  0.0522  0.0437  0.0180  0.0292  0.0351  0.1171  0.1257  0.1178  0.1258   
205  0.0303  0.0353  0.0490  0.0608  0.0167  0.1354  0.1465  0.1123  0.1945   
206  0.0260  0.0363  0.0136  0.0272  0.0214  0.0338  0.0655  0.1400  0.1843   

     0.2111  ...  0.0027  0.0065  0.0159  0.0072  0.0167  0.0180  0.0084  \
0    0.2872  ...  0.0084  0.0089  0.0048  0.0094  0.0191  0.0140  0.0049   
1    0.6194  ...  0.0232  0.0166  0.0095  0.0180  0.0244  0.0316  0.0164   
2    0.1264  ...  0.0121  0.0036  0.0150  0.0085  0.0073  0.0050  0.0044   
3    0.4459  ...  0.0031  0.0054  0.0105  0.0110  0.0015  0.0072  0.0048   
4    0.3039  ...  0.0045  0.0014  0.0038  0.0013  0.0089  0.0057  0.0027   
..      ...  ...     ...     ...     ...     ...     ...     ...     ...   
202  0.2684  ...  0.0116  0.0098  0.0199  0.0033  0.0101  0.0065  0.0115   
203  0.2154  ...  0.0061  0.0093  0.0135  0.0063  0.0063  0.0034  0.0032   
204  0.2529  ...  0.0160  0.0029  0.0051  0.0062  0.0089  0.0140  0.0138   
205  0.2354  ...  0.0086  0.0046  0.0126  0.0036  0.0035  0.0034  0.0079   
206  0.2354  ...  0.0146  0.0129  0.0047  0.0039  0.0061  0.0040  0.0036   

     0.0090  0.0032  R  
0    0.0052  0.0044  R  
1    0.0095  0.0078  R  
2    0.0040  0.0117  R  
3    0.0107  0.0094  R  
4    0.0051  0.0062  R  
..      ...     ... ..  
202  0.0193  0.0157  M  
203  0.0062  0.0067  M  
204  0.0077  0.0031  M  
205  0.0036  0.0048  M  
206  0.0061  0.0115  M  

[207 rows x 61 columns]

In [75]:
x=data.iloc[:,:-1].values
x.shape

(207, 60)

In [76]:
y=data.iloc[:,-1].values
y.shape

(207,)

In [77]:
encoder = LabelEncoder()
encoder.fit(y)
y = encoder.transform(y)
y

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0])

In [78]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

In [79]:
from tensorflow import keras
from tensorflow.keras.layers import Dense

Normal Backpropagation with Adam as optimizer

In [80]:
model=keras.Sequential()
model.add(Dense(60, input_shape=(60,), activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(1, activation='relu'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_60 (Dense)            (None, 60)                3660      
                                                                 
 dense_61 (Dense)            (None, 32)                1952      
                                                                 
 dense_62 (Dense)            (None, 64)                2112      
                                                                 
 dense_63 (Dense)            (None, 1)                 65        
                                                                 
Total params: 7,789
Trainable params: 7,789
Non-trainable params: 0
_________________________________________________________________


In [81]:
start=time.time()
model.fit(X_train,y_train,epochs=200)
end=time.time()

Epoch 1/200
5/5 [==============================] - 1s 4ms/step - loss: 1.6367 - accuracy: 0.5217
Epoch 2/200
5/5 [==============================] - 0s 3ms/step - loss: 0.7941 - accuracy: 0.5217
Epoch 3/200
5/5 [==============================] - 0s 3ms/step - loss: 0.7374 - accuracy: 0.5145
Epoch 4/200
5/5 [==============================] - 0s 3ms/step - loss: 0.7241 - accuracy: 0.3986
Epoch 5/200
5/5 [==============================] - 0s 3ms/step - loss: 0.7116 - accuracy: 0.3841
Epoch 6/200
5/5 [==============================] - 0s 3ms/step - loss: 0.6981 - accuracy: 0.4638
Epoch 7/200
5/5 [==============================] - 0s 3ms/step - loss: 0.6850 - accuracy: 0.5580
Epoch 8/200
5/5 [==============================] - 0s 3ms/step - loss: 0.6736 - accuracy: 0.5797
Epoch 9/200
5/5 [==============================] - 0s 3ms/step - loss: 0.6642 - accuracy: 0.6232
Epoch 10/200
5/5 [==============================] - 0s 3ms/step - loss: 0.6561 - accuracy: 0.6667
Epoch 11/200
5/5 [===========

In [85]:
print('Time required : ',end-start)
pred=model.predict(X_test)

Time required :  5.785518407821655
3/3 [==============================] - 0s 3ms/step


In [55]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, pred)

0.5652173913043478

L1 regularization

In [59]:
from keras import regularizers
model=keras.Sequential()
model.add(Dense(60, input_shape=(60,), activation='relu'))
model.add(Dense(32,activation='relu',kernel_regularizer=regularizers.l1(0.01)))
model.add(Dense(64,activation='relu',kernel_regularizer=regularizers.l1(0.01)))
model.add(Dense(1, activation='relu'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_36 (Dense)            (None, 60)                3660      
                                                                 
 dense_37 (Dense)            (None, 32)                1952      
                                                                 
 dense_38 (Dense)            (None, 64)                2112      
                                                                 
 dense_39 (Dense)            (None, 1)                 65        
                                                                 
Total params: 7,789
Trainable params: 7,789
Non-trainable params: 0
_________________________________________________________________


In [86]:
start=time.time()
model.fit(X_train,y_train,epochs=200)
end=time.time()

Epoch 1/200
5/5 [==============================] - 0s 8ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 2/200
5/5 [==============================] - 0s 6ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 3/200
5/5 [==============================] - 0s 6ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 4/200
5/5 [==============================] - 0s 8ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 5/200
5/5 [==============================] - 0s 6ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 6/200
5/5 [==============================] - 0s 4ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 7/200
5/5 [==============================] - 0s 6ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 8/200
5/5 [==============================] - 0s 5ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 9/200
5/5 [==============================] - 0s 7ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 10/200
5/5 [==============================] - 0s 5ms/step - loss: 0.0000e+00 - accura

In [87]:
print('Time required : ',end-start)

Time required :  5.562929391860962


L2 regularization

In [61]:
from keras import regularizers
model=keras.Sequential()
model.add(Dense(60, input_shape=(60,), activation='relu'))
model.add(Dense(32,activation='relu',kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(64,activation='relu',kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(1, activation='relu'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_40 (Dense)            (None, 60)                3660      
                                                                 
 dense_41 (Dense)            (None, 32)                1952      
                                                                 
 dense_42 (Dense)            (None, 64)                2112      
                                                                 
 dense_43 (Dense)            (None, 1)                 65        
                                                                 
Total params: 7,789
Trainable params: 7,789
Non-trainable params: 0
_________________________________________________________________


In [88]:
start=time.time()
model.fit(X_train,y_train,epochs=200)
end=time.time()

Epoch 1/200
5/5 [==============================] - 0s 6ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 2/200
5/5 [==============================] - 0s 3ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 3/200
5/5 [==============================] - 0s 3ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 4/200
5/5 [==============================] - 0s 4ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 5/200
5/5 [==============================] - 0s 3ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 6/200
5/5 [==============================] - 0s 3ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 7/200
5/5 [==============================] - 0s 3ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 8/200
5/5 [==============================] - 0s 3ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 9/200
5/5 [==============================] - 0s 4ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 10/200
5/5 [==============================] - 0s 3ms/step - loss: 0.0000e+00 - accura

In [89]:
print('Time required : ',end-start)

Time required :  10.329333066940308


Early Stopping

In [67]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience=5)

In [68]:
from keras import regularizers
model=keras.Sequential()
model.add(Dense(60, input_shape=(60,), activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(1, activation='relu'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_52 (Dense)            (None, 60)                3660      
                                                                 
 dense_53 (Dense)            (None, 32)                1952      
                                                                 
 dense_54 (Dense)            (None, 64)                2112      
                                                                 
 dense_55 (Dense)            (None, 1)                 65        
                                                                 
Total params: 7,789
Trainable params: 7,789
Non-trainable params: 0
_________________________________________________________________


In [90]:
start=time.time()
model.fit(
    X_train, 
    y_train, 
    epochs=200, 
    callbacks=[early_stopping],
    validation_data=(X_test, y_test)
)
end=time.time()

Epoch 1/200
5/5 [==============================] - 1s 169ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 1.9843 - val_accuracy: 0.8261
Epoch 2/200
5/5 [==============================] - 0s 30ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 1.9843 - val_accuracy: 0.8261
Epoch 3/200
5/5 [==============================] - 0s 32ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 1.9843 - val_accuracy: 0.8261
Epoch 4/200
5/5 [==============================] - 0s 32ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 1.9843 - val_accuracy: 0.8261
Epoch 5/200
5/5 [==============================] - 0s 29ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 1.9843 - val_accuracy: 0.8261
Epoch 6/200
5/5 [==============================] - 0s 15ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 1.9843 - val_accuracy: 0.8261


In [91]:
print('Time required : ',end-start)

Time required :  1.538851261138916


Dropout

In [92]:
from keras import regularizers
from tensorflow.keras.layers import Dropout
model=keras.Sequential()
model.add(Dense(60, input_shape=(60,), activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='relu'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_64 (Dense)            (None, 60)                3660      
                                                                 
 dense_65 (Dense)            (None, 32)                1952      
                                                                 
 dropout_2 (Dropout)         (None, 32)                0         
                                                                 
 dense_66 (Dense)            (None, 64)                2112      
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_67 (Dense)            (None, 1)                 65        
                                                                 
Total params: 7,789
Trainable params: 7,789
Non-train

In [93]:
start=time.time()
model.fit(X_train,y_train,epochs=200)
end=time.time()

Epoch 1/200
5/5 [==============================] - 2s 4ms/step - loss: 6.6962 - accuracy: 0.5217
Epoch 2/200
5/5 [==============================] - 0s 4ms/step - loss: 3.8940 - accuracy: 0.5217
Epoch 3/200
5/5 [==============================] - 0s 4ms/step - loss: 1.4789 - accuracy: 0.5145
Epoch 4/200
5/5 [==============================] - 0s 4ms/step - loss: 1.0547 - accuracy: 0.4928
Epoch 5/200
5/5 [==============================] - 0s 4ms/step - loss: 0.8868 - accuracy: 0.4638
Epoch 6/200
5/5 [==============================] - 0s 4ms/step - loss: 0.9020 - accuracy: 0.4565
Epoch 7/200
5/5 [==============================] - 0s 4ms/step - loss: 0.8012 - accuracy: 0.5652
Epoch 8/200
5/5 [==============================] - 0s 5ms/step - loss: 0.7448 - accuracy: 0.5725
Epoch 9/200
5/5 [==============================] - 0s 4ms/step - loss: 0.9567 - accuracy: 0.5580
Epoch 10/200
5/5 [==============================] - 0s 4ms/step - loss: 0.9351 - accuracy: 0.5000
Epoch 11/200
5/5 [===========

In [95]:
print('Time required : ',end-start)

Time required :  12.860960960388184
